In [ ]:
#pip install pandasql 

# Go to OHDSI Athena 網站下載 OHDSI vocabulary: ICD10CM and SNOMED
# https://athena.ohdsi.org/vocabulary/list

###  Use to produce ICD10 CM to SNOMED CT codes
#### SNOMED CT versioin: 01-Feb-2024
#### ICD10 CM version: 01-Oct-2024

In [27]:
import pandas as pd
import pandasql as psql

# Read CSV files into DataFrames

concept_df = pd.read_csv(r"C:\Users\wuche\Downloads\ICD_SNOMED\CONCEPT.csv", sep="\t")  # concept table

concept_rel_df = pd.read_csv(r"C:\Users\wuche\Downloads\ICD_SNOMED\concept_relationship.csv", sep="\t")  # concept_relationship table

# Run an SQL query on two DataFrames
query= """
SELECT 
    source.concept_code AS icd10_code, 
    source.concept_name AS icd10_name, 
    target.concept_code AS snomed_code, 
    target.concept_name AS snomed_name
FROM 
    concept_df AS source
INNER JOIN 
    concept_rel_df AS cr ON source.concept_id = cr.concept_id_1
INNER JOIN 
    concept_df AS target ON cr.concept_id_2 = target.concept_id
WHERE 
     source.vocabulary_id = 'ICD10CM' 
     and target.vocabulary_id = 'SNOMED'
     and cr.relationship_id = 'Maps to'
     and cr.invalid_reason IS NULL

"""
print("start")
result_df = psql.sqldf(query, locals())  # Execute SQL on DataFrame

# Display result
# for index, row in result_df.iterrows():
#     print(f"Row {index}: ICD10_CODE={row['icd10_code']}, ICD10_NAME={row['icd10_name']}, SNOMED_CODE={row['snomed_code']}, SNOMED_NAME={row['snomed_name']}")

result_df.to_csv("output_ICD10.csv", index=False)  # Save without index

# Unique ICD code count
unique_icd_count = result_df['icd10_code'].nunique()

# Count total records
total_records = result_df.shape[0]

# Print results
print(f"Unique ICD-10 Codes: {unique_icd_count}")
print(f"Total Records: {total_records}")

print("Done!") #97439 unique ICD 10 and Total record count: 128055
               #17263 unique ICD 9 and Total record count: 19691

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5,6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


start
Unique ICD-10 Codes: 97439
Total Records: 128055
Done!


### Compared with existing ICD10CM codes from TW 健保署

In [28]:
import pandas as pd
import pandasql as psql

# Read CSV anc XLSX files into DataFrames

file_path = "2024_ICD10CM_TW.xlsx"  # Change this to your file path
sheet_name = "ICD-10-CM"  # Change this to the sheet you want to read

# Read the Excel file into a DataFrame
TW_ICD_df = pd.read_excel(file_path, sheet_name=sheet_name)

filtered_df = TW_ICD_df[TW_ICD_df['USE'] == 1]

ICD_SNOMED_df = pd.read_csv(r"C:\Users\wuche\Downloads\output_ICD10.csv", delimiter=",")  # concept table
filtered_df.rename(columns={'2023年版\nICD-10-CM': 'ICD10CM_code'}, inplace=True)

# Run an SQL query on two DataFrames
query= """
SELECT *
FROM 
    filtered_df

WHERE 
     ICD10CM_code NOT in (select icd10_code from ICD_SNOMED_df)
    

"""
print("start")
result_df = psql.sqldf(query, locals())  # Execute SQL on DataFrame


# Count total records
total_records = result_df.shape[0]

# Unique ICD code count
unique_icd_count = result_df['ICD10CM_code'].nunique()

unique_tw_icd_count = filtered_df['ICD10CM_code'].nunique()
# Count total records
total_records = result_df.shape[0]

# Print results
print(f"Unique TW ICD-10 Codes: {unique_tw_icd_count}")
print(f"Unique ICD-10 Codes not in ICD10_SNOMED_CT table: {unique_icd_count}")

print("Done!") # 73681 unique ICD10 CODEs from TW dataset. But, 707 沒有在 ICD10_SNOMEDCT 對照檔中

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


start
Unique TW ICD-10 Codes: 73681
Unique ICD-10 Codes not in ICD10_SNOMED_CT table: 707
Done!
